In [3]:
import numpy as np
import bokeh as bk
from bokeh.io import push_notebook
import bokeh.plotting as bkp
import bokeh.models as bkm
from bokeh.io import push_notebook
from ipywidgets import interact
import ipywidgets as widgets
bkp.output_notebook()

Loading BokehJS ...

# Analytical model for spherical sludge growth in Al-Fe system

## References:

1. An Analytical Solidification Model for Solute Redistribution during of Planar, Columnar, or Equiaxed Morphology L. Nastac and D.M. Stefanescu: Metall. Trans. A, 1993, vol. 24A, pp. 2107-18

2. Computational Modeling of NbC/Laves Formation In INCONEL 718 Equiaxed Castings L. Nastac and D.M. Stefanescu: Metall. Mater. Trans. A, 1997, vol. 28A, pp. 1582-1587

# Introduction:

## Goal: 

Application of Nastac-Stefanescu model for morphology of sludge ($ \beta $ aluminum) particles

## Assumptions:

<ul>
    <li>Instantaneous nucleation</li>
    <li>Sludge growth only in liquid</li>
    <li>negligible interference between growing
sludge</li>
    <li>volume diffusion limited growth of sludge.</li>
    <li>Constant growth velocity</li>
    
</ul>

## Schematic:

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/22/nucleus.png" /img>

<strong> Figure 1. Schematic of volume element (from ref.1 ) </strong>

# Calculation strategy:

<ol>
    <li> Determine the material properties of Al </li>
    <li> Determine the material properties of Fe in Al </li>
    <li> Heat transfer: Nova </li>
    <li> Determine final solidification time </li>
    <li> Experimentally determine final sludge particle radius as function of cooling rate </li>
    <li> calculate interface growth velocity: </li>
    <li> $$ V_{\beta} = \frac{\Omega_{Fe}}{\Phi_{\beta}} \frac{D_L}{R_D-R_f} $$ </li>
    <li> $$ \Phi_{\beta} = \frac{4 \pi R_{\beta}}{A_s} $$ </li>
    <li> $$ R_D = \frac{3}{4\pi} \Bigg( \frac{v_l}{N_{\beta}} \Bigg)^{1/3} $$ </li>
    <li> $$ \Omega_{Fe} = \frac{C_L^* - C_0}{C_L^*-C_S^*} $$ </li>
    <li> Calculate interface radius: </li>
    <li> $$ R^* = V_{\beta}t $$ </li>
    <li> calculate f raction of solid: </li>
    <li> $$ f_s=(R^*/Rf)^3 $$</li>
    <li> Calculate interface liquid iron concentration as function of fraction of solid: </li>
    <li> $$ C_L^*= \frac{C_0\big( kI_S +I_L -\frac{1}{3} \big) + Q }{kI_S + I_L - \frac{1}{3} + \frac{1-k f_S}{3}} $$</li>
    <li> $$ Q= \frac{v_{\beta}}{v_{\Omega}}\frac{\rho_{\beta}}{\rho_{eu}} pct \; Fe_\beta  $$ </li>
    <li> $$ I_S = \frac{2f_S}{\pi^2} \sum_{n=1}^{\infty} \frac{1}{n^2} \Bigg[-\big( \frac{n \pi}{f_s^{1/3}} \big) \frac{D_St}{R^2_f} \Bigg]  $$ </li>
    <li> $$ I_L = 2f_s^{2/3}(1-f_s^{1/3}) \sum_{n=1}^{\infty} \frac{1}{\alpha^2_n} exp \Bigg[ -\Big( \frac{\alpha_n}{1/f_s^{1/3}} \Big)^2 \frac{D_L t}{R^2_f} \Bigg] $$ </li>
    <li> $$ \alpha_n R_f  \cot {\alpha_n} = R_f - R^* $$ </li>
    <li> Calculate sludge content (wt%) as function of fraction of solid </li>
</ol>

# Implementation:

## Experimental sludge size:

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/24/sludge.png" />

## Cooling rate:

In [7]:
#Cooling rate, K/S
cooling_rate=np.array([1.2,1.7,2.8,10,25])
print(cooling_rate)

[  1.2   1.7   2.8  10.   25. ]


## Rf:

In [9]:
#Sludge area, um^2
sludge_area=np.array([380,290,240,235,245])
#Sludge area, m^2
sludge_area=sludge_area*10**(-12)
#Sludge radius, m
sludge_radius=np.sqrt(sludge_area)
print(sludge_radius)

[  1.94935887e-05   1.70293864e-05   1.54919334e-05   1.53297097e-05
   1.56524758e-05]


## Iteration:

In [24]:
cr=cooling_rate[0]
R_f=sludge_radius[0]
t_f=20
C_0=10.127
k=0.1539
N_beta=5
v_l=3/4*np.pi*R_f**3
D_L=2.69*10**(-9)
delta_t=1
pct_Fe_beta=18.15
rho_beta=2300
rho_eut=2300

R_D=(3/(4*np.pi))*(v_l/N_beta)**(1/3)


def sphere_volume(R):
    return (3/4*np.pi*R**3)

def sol_sup_sat(C_l):
    return (C_l-C_0)/(C_l-C_l/k)
    
def phi(R_beta,A_beta):
    return (4*np.pi*R_beta/A_beta)

def V_calc(C_l,R_beta,A_beta):
    return sol_sup_sat(C_l)/phi(R_beta,A_beta)*D_L/(R_D-R_beta)

def Q(R_beta):
    return sphere_colume(R_beta)/v_l*(rho_beta/rho_eut)

def I_S():

C_l=np.zeros(t_f)
V_beta=np.zeros(t_f)
R_beta=np.zeros(t_f)
A_beta=np.zeros(t_f)
f_s=np.zeros(t_f)

C_l[0]=C_0


V_beta[0]=V_calc(C_l[0],R_beta[0],A_beta[0])

R_beta[1]=R_beta[0]+V_beta[0]*delta_t
f_s[1]=(R_beta[1]/R_f)**3






    

for i in range(0,t_f,1):
    break
    
print(C_l_star)
    

[ 10.127   0.      0.      0.      0.      0.      0.      0.      0.      0.
   0.      0.      0.      0.      0.      0.      0.      0.      0.      0.   ]
